In [1]:
from empath import Empath
lexicon = Empath()
import pandas as pd
import csv
import numpy as np


In [2]:

raw_news1 = pd.read_csv("articles1.csv")
raw_news2 = pd.read_csv("articles2.csv")
raw_news3 = pd.read_csv("articles3.csv")

print(raw_news1.shape[0])

50000


In [3]:
def change_date_expression(date):
    if date[4] == "-":
        return date
    
    break_point = []
    for i in range(len(date)):
        if date[i] == '/':
            break_point.append(i)
    
    if len(break_point) != 0:
        out = date[0:break_point[0]] + '-' + date[break_point[0]+1:break_point[1]]+'-'+date[break_point[1]+1:]
        return out
    else:
        return date

In [4]:
def count_day(dataset,dict_count):
    for i in range(dataset.shape[0]):
        date = dataset[i][5];
        if dict_count.__contains__(date):
            dict_count[date] += 1
        else:
            dict_count[date] = 1

In [5]:
def relevance_caculation(dic):
    relavences = ["social_media","computer","business","programming","hearing","urban","shopping","science","work","valuable","fashion","technology","competing","economics","office"]
    negetive_attitude = ['hate','aggression','horror','suffering','ridicule','irritability','deception','disappointment','negative_emotion','nervousness']
    positive_attitude = ['cheerfulness','optimism','celebration','trust','positive_emotion']
    
    # correlation c:
    c = 0
    for relavence in relavences:
        c += dic[relavence]
    c = float(c) / float(len(relavences))
    
    # negetive attitude:
    neg_att = 0
    for neg in negetive_attitude:
        neg_att += dic[neg]
    neg_att = float(neg_att) / float(len(negetive_attitude))
    
    # positive attitude:
    pos_att = 0
    for pos in positive_attitude:
        pos_att += dic[pos]
    pos_att = float(neg_att) / float(len(positive_attitude))
    
    return float(c*100), float(c*neg_att*1000), float(c*pos_att*1000)

In [6]:
def pre_processing_dataset(dataset):
    # first we need to drop and add some columns
    # the colums we need to drop is "id", "title", "publication", "author", "year", "month", "url", "Unnamed: 0"
    dataset = dataset.drop(columns = ["id", "title", "publication", "author", "year", "month", "url", "Unnamed: 0"])
    dataset["correlation"] = 0
    dataset["positive_relavent"] = 0
    dataset["negetive_relavent"] = 0
    
    dataset["correlation"] = dataset.correlation.astype(float)
    dataset["positive_relavent"] = dataset.positive_relavent.astype(float)
    dataset["negetive_relavent"] = dataset.negetive_relavent.astype(float)
    return dataset

In [7]:
def processing_analysis_with_only_three_value(dataset):
    # the colums we need to add is sentiment analysis result 
    # plus three special colums, which are correlation c, positive attitude p * c, negetive attitude n *c.
    # second we need to change the all the date style into 2017-1-1
    # all this need to do in a loop:
    for i in range(dataset.shape[0]):
        # change date:
        if pd.isna(dataset["date"][i]) == True:
            print(dataset["date"][i])
        else:
            dataset["date"][i] = change_date_expression(dataset["date"][i])
        
        # analysis the contont:
        ana_dict = lexicon.analyze(dataset["content"][i], normalize = True)
        #for key, value in ana_dict.items():
            #dataset[key][i] = value
        # puls three special values.
        if ana_dict is None:
            dataset["correlation"][i] = 0
            dataset["positive_relavent"][i] = 0
            dataset["negetive_relavent"][i] = 0
        else:
            c, neg, pos = relevance_caculation(ana_dict)
            dataset["correlation"][i] = c
            dataset["positive_relavent"][i] = neg
            dataset["negetive_relavent"][i] = pos
        
        if i % 1000 == 0:
            print(i)
            print(dataset["date"][i])
        
    return dataset 

In [8]:
def out_put_result(dataframe, n):
    file_name = str(n)+"_part_anaylsis.csv"
    
    dataframe1 = pre_processing_dataset(dataframe)
    dataframe1_with_analysis = processing_analysis(dataframe1)
    dataframe1_with_analysis = dataframe1_with_analysis.drop(columns = ["content"])
    
    dataframe1_with_analysis.to_csv(file_name, encoding='utf-8')
    
    return dataframe1_with_analysis

In [12]:
# ["social_media","computer","business","programming","hearing","urban","shopping","science","work",
#      "valuable","fashion","technology","competing","economics","office"]
# ['hate','aggression','horror','suffering','ridicule','irritability','deception',
#      'disappointment','negative_emotion','nervousness']
# ['cheerfulness','optimism','celebration','trust','positive_emotion']
# now what i should do is keep all this values
# so need dataframe with columns: "date", ""social_media","computer","business","programming","hearing",
#       "urban","shopping","science","work",
#      "valuable","fashion","technology","competing","economics","office", 'hate','aggression',
#       'horror','suffering','ridicule','irritability','deception',
#      'disappointment','negative_emotion','nervousness', 'cheerfulness','optimism','celebration','trust',
#      'positive_emotion',

In [22]:
def processing_analysis_with_all_variables(dataset):
    # pre-processing the dataset for droping useless colums
    dataset = dataset.drop(columns = ["id", "title", "publication", "author", "year", "month", "url", "Unnamed: 0"])
    # we need to have a list hold all the parameters
    check_list = ["social_media","computer","business","programming","hearing","urban","shopping","science","work","valuable","fashion","technology","competing","economics","office",'cheerfulness','optimism','celebration','trust','positive_emotion','hate','aggression','horror','suffering','ridicule','irritability','deception', 'disappointment','negative_emotion','nervousness']    
    # inital matrix that have all the columens:
    data_list = ["date-month-day"]
    for parameter in check_list:
        data_list.append(np.float16(0))
    
    for i in range(3):
        data_list.append(np.float16(0))
        
    data_matrix = np.array([data_list])
    
    print(data_matrix)
    
    for i in range(dataset.shape[0]):
        # change date:
        if pd.isna(dataset["date"][i]) == True:
            print(dataset["date"][i])
        else:
            dataset["date"][i] = change_date_expression(dataset["date"][i])
        
        # analysis the contont:
        ana_dict = lexicon.analyze(dataset["content"][i], normalize = True)
        #for key, value in ana_dict.items():
            #dataset[key][i] = value
        # puls three special values.
        
        if ana_dict is not None:
            one_line_list = []
            one_line_list.append(dataset["date"][i])
            for parameter in check_list:
                one_line_list.append(ana_dict[parameter])
            relev, pos, neg = relevance_caculation(ana_dict)
            one_line_list.append(relev)
            one_line_list.append(pos)
            one_line_list.append(neg)
            
            data_matrix = np.insert(data_matrix, 0, [one_line_list], 0)
        
        if i % 1000 == 0:
            print(i)
            print(dataset["date"][i])
    
    dataframe = pd.DataFrame(data_matrix, columns=['date', "social_media","computer","business","programming","hearing","urban","shopping","science","work","valuable","fashion","technology","competing","economics","office",'cheerfulness','optimism','celebration','trust','positive_emotion','hate','aggression','horror','suffering','ridicule','irritability','deception', 'disappointment','negative_emotion','nervousness' ,'correlation', 'positive_relavent','negetive_relavent'])

    return dataframe



In [23]:
def get_final_result_into_csv(dataframe, n):
    file_name = str(n)+"_part_anaylsis_with_all_parameters.csv"
    
    dataframe1_with_analysis = processing_analysis_with_all_variables(dataframe)
    
    dataframe1_with_analysis.to_csv(file_name, encoding='utf-8')
    
    return dataframe1_with_analysis

In [ ]:
news1 = get_final_result_into_csv(raw_news1,1)

news1.head(10)

news2 = get_final_result_into_csv(raw_news2,2)

news2.head(10)

news3 = get_final_result_into_csv(raw_news3,3)

news3.head(10)